## Tweets Disaster 

In [2]:
import pandas as pd

train_dataset = pd.read_csv('/media/mahfuz/Media/work/tweets_disaster/nlp-getting-started/train.csv')
train_dataset.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Find the null value in the dataset
print(train_dataset.isnull().sum())

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64


In [6]:
# select the test dataset
test_dataset = pd.read_csv('/media/mahfuz/Media/work/tweets_disaster/nlp-getting-started/test.csv')
test_dataset.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
print('Train shape: ', train_dataset.shape)
print('Test shape: ', test_dataset.shape)
print('Column Names: ', train_dataset.columns)

Train shape:  (7613, 5)
Test shape:  (3263, 4)
Column Names:  Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')


In [10]:
train_dataset_final = train_dataset[['text', 'target']]
train_dataset_final.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [11]:
train_dataset_final.to_csv('final_dataset.csv', index=False)